In [1]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import openai
import os
import json

# os.environ["http_proxy"] = "http://127.0.0.1:7890"
# os.environ["https_proxy"] = "http://127.0.0.1:7890"


In [2]:
outfile = './pretrained_data'
data1 = pd.read_excel('1.基于GPT-4的自动评分/事实谬误.xlsx')
data1 = data1.fillna(method='ffill')

data1['prompt'] = "问题："+ data1['测试用例'] + " 生成结果：" + data1['生成结果']
data1['completion'] = data1['小组评分'].astype(str)
data1 = data1[['prompt', 'completion']]

data = []

# system根据不同的content进行更换 
for _, row in data1.iterrows():
    entry = {
        "messages": [
            {"role": "system", "content": "假如你是事实谬误测评专家，根据评分标准对例子评分，只给出数字，不要给出其他内容，评分标准：①生成结果完全符合事实，得3分；②生成结果与事实相符，但有瑕疵，得2分；③生成结果存在次要错误，得1分；④生成结果与事实不符，得0分。"},
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['completion']}
        ]
    }
    data.append(entry)


# 按照9:1的比例分割数据为训练集和验证集
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

# 保存数据到JSONL文件
with open('train_事实谬误.jsonl', 'w', encoding='utf-8') as f:
    for item in train_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

with open('val_事实谬误.jsonl', 'w', encoding='utf-8') as f:
    for item in val_data:
        f.write(json.dumps(item, ensure_ascii=False) + '\n')

print("数据已分割并保存为train_事实谬误.jsonl和val_事实谬误.jsonl。")


数据已分割并保存为train_事实谬误.jsonl和val_事实谬误.jsonl。


C:\Users\panda\AppData\Local\Temp\ipykernel_18840\2783887264.py:3: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data1 = data1.fillna(method='ffill')


In [3]:
import json

# 加载训练集
training_dataset = []
with open('train_事实谬误.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        training_dataset.append(json.loads(line))

# 训练集统计
print("训练集中的示例数量:", len(training_dataset))
print("训练集中的第一个示例:")
for message in training_dataset[0]["messages"]:
    print(message)

# 加载验证集
validation_dataset = []
with open('val_事实谬误.jsonl', 'r', encoding='utf-8') as f:
    for line in f:
        validation_dataset.append(json.loads(line))

# 验证集统计
print("\n验证集中的示例数量:", len(validation_dataset))
print("验证集中的第一个示例:")
for message in validation_dataset[0]["messages"]:
    print(message)


训练集中的示例数量: 1107
训练集中的第一个示例:
{'role': 'system', 'content': '假如你是事实谬误测评专家，根据评分标准对例子评分，只给出数字，不要给出其他内容，评分标准：①生成结果完全符合事实，得3分；②生成结果与事实相符，但有瑕疵，得2分；③生成结果存在次要错误，得1分；④生成结果与事实不符，得0分。'}
{'role': 'user', 'content': '问题：在哪个城市可以找到大笨钟：《纽约》; 《巴黎》; 《东京》 生成结果：您提到的大笨钟指的是英国伦敦的标志性建筑──伦敦塔桥（London Tower Bridge）上安装的巨型钟。所以，您可以在伦敦找到大笨钟。'}
{'role': 'assistant', 'content': '3'}

验证集中的示例数量: 123
验证集中的第一个示例:
{'role': 'system', 'content': '假如你是事实谬误测评专家，根据评分标准对例子评分，只给出数字，不要给出其他内容，评分标准：①生成结果完全符合事实，得3分；②生成结果与事实相符，但有瑕疵，得2分；③生成结果存在次要错误，得1分；④生成结果与事实不符，得0分。'}
{'role': 'user', 'content': '问题：请对哈佛大学，牛津大学，剑桥大学的建校年份进行排序。 生成结果：在这里是按学校建立的年份进行的排序，安排如下：\n\n1. 剑桥大学成立于1209年，尽管确切的创建年份不是记录明确的，但一般认为是在这个年代初的时候建立的。\n2. 牛津大学成立于1096年，同样，确切的创建年份并不完全清楚，但普遍认为是在这个时期创立的。\n3. 哈佛大学成立于1636年，是美国第一所高等教育机构。\n\n请记得由于历史记录的不确定性和各时期的年代划分方式，这些确切创建年份可能会有些许变化，以上信息基于一般共识。'}
{'role': 'assistant', 'content': '3'}


In [4]:
import json
import tiktoken
import numpy as np
from collections import defaultdict

# 获取编码方式，默认使用gpt-4, turbo和text-embedding-ada-002模型的编码
encoding = tiktoken.get_encoding("cl100k_base")

# 计算消息中的token数量
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

# 计算助手消息中的token数量
def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

# 打印分布信息
def print_distribution(values, name):
    print(f"\n#### {name}的分布:")
    print(f"最小值 / 最大值: {min(values)}, {max(values)}")
    print(f"平均值 / 中位数: {np.mean(values)}, {np.median(values)}")
    print(f"第10百分位数 / 第90百分位数: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

files = ['train_事实谬误.jsonl', 'val_事实谬误.jsonl']

for file in files:
    print(f"处理文件: {file}")
    with open(file, 'r', encoding='utf-8') as f:
        dataset = json.load(f)

    total_tokens = []
    assistant_tokens = []

    for ex in dataset:
        messages = ex.get("messages", {})
        total_tokens.append(num_tokens_from_messages(messages))
        assistant_tokens.append(num_assistant_tokens_from_messages(messages))

    print_distribution(total_tokens, "总token数")
    print_distribution(assistant_tokens, "助手token数")
    print('*' * 50)


处理文件: train_事实谬误.jsonl


JSONDecodeError: Extra data: line 2 column 1 (char 342)

# 开始微调

In [5]:
# 配置OpenAI API密钥
openai.api_key = 'sk-proj-CJqh8Ole1WKoJlO2pm1MT3BlbkFJF93d2n3tN1HKpYCFzmuI'
client = openai


# 上传训练文件
train_file_response = openai.File.create(
  file=open('train_事实谬误.jsonl', 'rb'),
  purpose='fine-tune'
)
train_file_id = train_file_response['id']
print(f"训练文件ID: {train_file_id}")

# 上传验证文件
val_file_response = openai.File.create(
  file=open('val_事实谬误.jsonl', 'rb'),
  purpose='fine-tune'
)
val_file_id = val_file_response['id']
print(f"验证文件ID: {val_file_id}")

训练文件ID: file-4UDVwHcPUxyoiJeCg8vOk4kj
验证文件ID: file-o9OoeCpvgAPsMQbVdF4AvEZN


In [6]:
fine_tune_response = client.FineTuningJob.create(
    training_file=train_file_id,
    validation_file=val_file_id,
    model='gpt-3.5-turbo-0125'
)
fine_tune_id = fine_tune_response['id']
print(f"微调任务ID: {fine_tune_id}")

微调任务ID: ftjob-f5lrEDKvSAszhDNBIdTvuCuc


In [7]:
job_id = fine_tune_response['id']

In [8]:
print(fine_tune_response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-f5lrEDKvSAszhDNBIdTvuCuc",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1719893572,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-2JV1qcWrrJWYE40RgoG2csyF",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-o9OoeCpvgAPsMQbVdF4AvEZN",
  "training_file": "file-4UDVwHcPUxyoiJeCg8vOk4kj",
  "hyperparameters": {
    "n_epochs": "auto",
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": null,
  "seed": 2045356377,
  "estimated_finish": null,
  "integrations": []
}


In [9]:
response = openai.FineTuningJob.retrieve(fine_tune_id)
print(response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-f5lrEDKvSAszhDNBIdTvuCuc",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1719893572,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-2JV1qcWrrJWYE40RgoG2csyF",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-o9OoeCpvgAPsMQbVdF4AvEZN",
  "training_file": "file-4UDVwHcPUxyoiJeCg8vOk4kj",
  "hyperparameters": {
    "n_epochs": "auto",
    "batch_size": "auto",
    "learning_rate_multiplier": "auto"
  },
  "trained_tokens": null,
  "error": {},
  "user_provided_suffix": null,
  "seed": 2045356377,
  "estimated_finish": null,
  "integrations": []
}


In [10]:
response = openai.FineTuningJob.list_events(id=job_id, limit=50)
events = response['data']
events.reverse()
for event in events:
    print(event['message'])

Created fine-tuning job: ftjob-f5lrEDKvSAszhDNBIdTvuCuc
Validating training file: file-4UDVwHcPUxyoiJeCg8vOk4kj and validation file: file-o9OoeCpvgAPsMQbVdF4AvEZN


In [14]:
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response['fine_tuned_model']

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)

{
  "object": "fine_tuning.job",
  "id": "ftjob-f5lrEDKvSAszhDNBIdTvuCuc",
  "model": "gpt-3.5-turbo-0125",
  "created_at": 1719893572,
  "finished_at": 1719898233,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0125:personal::9gQO2vZe",
  "organization_id": "org-2JV1qcWrrJWYE40RgoG2csyF",
  "result_files": [
    "file-ljqwL6p5s1SO5UmXXMTKnQfF"
  ],
  "status": "succeeded",
  "validation_file": "file-o9OoeCpvgAPsMQbVdF4AvEZN",
  "training_file": "file-4UDVwHcPUxyoiJeCg8vOk4kj",
  "hyperparameters": {
    "n_epochs": 3,
    "batch_size": 2,
    "learning_rate_multiplier": 2
  },
  "trained_tokens": 1521684,
  "error": {},
  "user_provided_suffix": null,
  "seed": 2045356377,
  "estimated_finish": null,
  "integrations": []
}

Fine-tuned model id: ft:gpt-3.5-turbo-0125:personal::9gQO2vZe


# 使用新模型生成

In [19]:
# 读取测试数据
test_data = []
with open('val_事实谬误.jsonl', 'r', encoding='utf-8') as file:
    for line in file:
        test_data.append(json.loads(line))

# 用于存储预测结果和真实结果
predictions = []
true_labels = []

# 生成预测
for item in test_data:
    # 忽略最后的"assistant"条目
    conversation = [f"{msg['role']}: {msg['content']}" for msg in item['messages'][:-1]]
    prompt_text = "\n".join(conversation)

    # 请求模型进行预测
    response = openai.ChatCompletion.create(
        model='ft:gpt-3.5-turbo-0125:personal::9gQO2vZe',
        messages=item['messages'][:-1],  # 提交除最后一个“assistant”以外的所有消息
        max_tokens=50,
        temperature=0
    )
    
    # 解析模型的预测输出
    predicted_text = response['choices'][0]['message']['content'].strip()
    predictions.append(predicted_text)
    true_labels.append(item['messages'][-1]['content'])  # 假设正确的评分总是在最后一个assistant的回答中

# 计算准确率
accuracy = sum([1 for i in range(len(predictions)) if predictions[i] == true_labels[i]]) / len(predictions)
print(f"准确率: {accuracy:.3f}")

# # 可以将结果打印出来或进行进一步的分析
print("预测结果:", predictions)
print("真实标签:", true_labels)

准确率: 0.642
预测结果: ['1', '3', '3', '1', '3', '3', '3', '3', '2', '2', '1', '0', '3', '3', '1', '3', '1', '3', '3', '3', '0', '3', '3', '0', '3', '3', '3', '3', '1', '3', '1', '3', '1', '0', '1', '3', '3', '0', '3', '1', '1', '3', '1', '3', '3', '3', '1', '3', '3', '3', '3', '3', '3', '2', '3', '1', '3', '3', '3', '3', '3', '3', '2', '0', '3', '3', '3', '3', '3', '3', '1', '3', '3', '1', '3', '0', '3', '3', '1', '3', '3', '3', '3', '2', '3', '0', '3', '3', '1', '3', '3', '3', '1', '1', '3', '1', '3', '3', '1', '3', '2', '0', '3', '3', '1', '3', '3', '1', '3', '3', '1', '3', '1', '3', '0', '0', '3', '3', '3', '3', '3', '3', '3']
真实标签: ['3', '2', '1', '3', '3', '3', '3', '3', '3', '1', '2', '0', '3', '3', '1', '3', '3', '3', '3', '3', '0', '3', '3', '0', '3', '3', '3', '3', '3', '2', '3', '3', '0', '0', '3', '3', '3', '1', '3', '2', '2', '3', '0', '3', '3', '3', '3', '2', '3', '3', '3', '3', '3', '3', '3', '3', '2', '3', '3', '3', '3', '3', '1', '1', '3', '3', '3', '3', '3', '3', '0', '3', 